In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_DLNN_CORENup_v2"
outPath = "Results"
foldName = "folds.pickle"

epochs = 100
batch_size = 32
shuffle = True
seed = None

input_data_folder = "Data"
training_data_file = "Training-datasets-PredNTS.txt"
independent_data_file = "independent dataset-PredNTS.txt"

In [2]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, matthews_corrcoef

import math

In [3]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
##################################################################################
##### define all CUSTOM functions
##################################################################################

def one_hot_encode_nt(sequence, char_dict):
    
    seq_encoded = np.zeros((len(sequence),len(char_dict)))
    
    i = 0
    for single_character in sequence:
        if(single_character.upper() in char_dict.keys()):
            seq_encoded[i][char_dict[single_character.upper()]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in NT sequence: '+sequence)
    return seq_encoded

In [5]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [6]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [7]:
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_CORENup(input_seq_shape = (41, 21),
                 conv_filters_per_layer_1 = 10, kernel_length_1 = 10, conv_strides_1 = 1, ## 1st Convolutional layer parameters
                 max_pool_width_1 = 3, max_pool_stride_1 = 3, ## 1st Maxpool layer parameters
                 lstm_decode_units = 25, ## LSTM layer parameters
                 conv_filters_per_layer_2 = 10,  kernel_length_2 = 5, conv_strides_2 = 1, ## 2nd Convolutional layer parameters
                 max_pool_width_2 = 3, max_pool_stride_2 = 3, ## 2nd Maxpool layer parameters
                 dense_decode_units = 128, ## Dense layer parameters
                 prob = 0.5, learn_rate = 0.0005, 
                 loss = 'binary_crossentropy', metrics = None):
    
    beta = 0.001
    
    ######################################################################################################
    ########  SEQUENCE  ##################################################################################
    ######################################################################################################
    
    input1 = tf.keras.layers.Input(shape=input_seq_shape)

    x1 = tf.keras.layers.Conv1D(conv_filters_per_layer_1, kernel_length_1,
                                strides = conv_strides_1, kernel_regularizer = tf.keras.regularizers.l2(beta), 
                                padding = "same")(input1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    x1 = tf.keras.layers.MaxPool1D(pool_size = max_pool_width_1, strides = max_pool_stride_1)(x1)
    x1 = tf.keras.layers.Dropout(prob)(x1)

    ## LSTM Path

    x2 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta))(x1)
    
    x2 = tf.keras.layers.Dropout(prob)(x2)
    
    x2 = tf.keras.layers.Flatten()(x2)

    ## Conv Path

    x3 = tf.keras.layers.Conv1D(conv_filters_per_layer_2, kernel_length_2, strides = conv_strides_2, 
                                kernel_regularizer = tf.keras.regularizers.l2(beta), padding = 'same')(x1)
    x3 = tf.keras.layers.Activation('relu')(x3)
    x3 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width_2, strides = max_pool_stride_2)(x3)
    x3 = tf.keras.layers.Dropout(prob)(x3)
    
    x3 = tf.keras.layers.Flatten()(x3)
    
    x4 = tf.keras.layers.Concatenate(1)([x2,x3])
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu'
                             )(x4)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), loss = loss)

    return model

In [8]:
# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_CORENup(input_seq_shape = (41, 21),
#                  conv_filters_per_layer_1 = 10, kernel_length_1 = 10, conv_strides_1 = 1, ## 1st Convolutional layer parameters
#                  max_pool_width_1 = 3, max_pool_stride_1 = 3, ## 1st Maxpool layer parameters
#                  lstm_decode_units = 15, ## LSTM layer parameters
#                  conv_filters_per_layer_2 = 10,  kernel_length_2 = 5, conv_strides_2 = 1, ## 2nd Convolutional layer parameters
#                  max_pool_width_2 = 3, max_pool_stride_2 = 3, ## 2nd Maxpool layer parameters
#                  dense_decode_units = 32, ## Dense layer parameters
#                  prob = 0.5, learn_rate = 0.0005, 
#                  loss = 'binary_crossentropy', metrics = None):

# # def DLNN_CORENup(input_seq_shape = (41, 21),
# #                  conv_filters_per_layer_1 = 10, kernel_length_1 = 10, conv_strides_1 = 1, ## 1st Convolutional layer parameters
# #                  max_pool_width_1 = 3, max_pool_stride_1 = 3, ## 1st Maxpool layer parameters
# #                  lstm_decode_units = 15, ## LSTM layer parameters
# #                  conv_filters_per_layer_2 = 10,  kernel_length_2 = 5, conv_strides_2 = 1, ## 2nd Convolutional layer parameters
# #                  max_pool_width_2 = 3, max_pool_stride_2 = 3, ## 2nd Maxpool layer parameters
# #                  dense_decode_units = 128, ## Dense layer parameters
# #                  prob = 0.75, learn_rate = 0.0005, 
# #                  loss = 'binary_crossentropy', metrics = None):

#     beta = 0.001
    
#     ######################################################################################################
#     ########  SEQUENCE  ##################################################################################
#     ######################################################################################################
    
#     input1 = tf.keras.layers.Input(shape=input_seq_shape)

#     x1 = tf.keras.layers.Conv1D(conv_filters_per_layer_1, kernel_length_1,
#                                 strides = conv_strides_1, kernel_regularizer = tf.keras.regularizers.l2(beta)
#                                )(input1)
#     x1 = tf.keras.layers.Activation('relu')(x1)
#     x1 = tf.keras.layers.MaxPool1D(pool_size = max_pool_width_1, strides = max_pool_stride_1)(x1)
#     x1 = tf.keras.layers.Dropout(prob)(x1)

#     ## LSTM Path
    
# #     x2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_decode_units, 
# #                                                             return_sequences = True, 
# #                                                             kernel_regularizer = tf.keras.regularizers.l2(beta)
# #                                                            )
# #                                       )(x1)

#     x2 = tf.keras.layers.GRU(lstm_decode_units, 
#                              kernel_regularizer = tf.keras.regularizers.l2(beta))(x1)
    
#     x2 = tf.keras.layers.Dropout(prob)(x2)
    
#     x2 = tf.keras.layers.Flatten()(x2)

#     ## Conv Path

#     x3 = tf.keras.layers.Conv1D(conv_filters_per_layer_2, kernel_length_2, strides = conv_strides_2, 
#                                 kernel_regularizer = tf.keras.regularizers.l2(beta)
#                                )(x1)
#     x3 = tf.keras.layers.Activation('relu')(x3)
#     x3 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width_2, strides = max_pool_stride_2)(x3)
#     x3 = tf.keras.layers.Dropout(prob)(x3)
    
#     x3 = tf.keras.layers.Flatten()(x3)
    
#     x4 = tf.keras.layers.Concatenate(1)([x2,x3])
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'relu')(x4)
    
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
# #     initial_learning_rate = 0.1
# #     decay_steps = 1.0
# #     decay_rate = 0.5
# #     learning_rate_fn = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate, decay_steps, decay_rate)

#     learning_rate_fn = tf.keras.optimizers.schedules.ExponentialDecay(
#         initial_learning_rate=1e-1,
#         decay_steps=10000,
#         decay_rate=0.9
#     )

#     opt = tf.keras.optimizers.SGD(learning_rate=learning_rate_fn)
    
#     ## Compile model
#     if(metrics != None):
# #         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), loss = loss, metrics = metrics)
# #         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn), loss = loss, metrics = metrics)
#         model.compile(optimizer = opt, loss = loss, metrics = metrics)
#     else:
# #         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), loss = loss)
#         model.compile(optimizer = opt, loss = loss)
    
#     return model

In [9]:
# for step in range(10):
#     initial_learning_rate=1e-1
#     decay_steps=10000
#     decay_rate=0.9
#     print(step, ':', initial_learning_rate * decay_rate ** (step / decay_steps))

In [10]:
DLNN_CORENup().summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 41, 21)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 41, 10)       2110        ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 41, 10)       0           ['conv1d[0][0]']                 
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 13, 10)       0           ['activation[0][0]']             
                                                                                              

# Training

In [11]:
##################################################################################
##### read training file
##################################################################################
train_file_path = os.path.join(input_data_folder, training_data_file)
train_data = pd.read_csv(train_file_path, sep='\t', header=None)
train_data.columns = ['Sequence', 'name', 'id', 'flag', 'label_original', 'type']
train_data.head()

##################################################################################
##### Create dictionary of all characters in the NT sequence 
##################################################################################
all_char_set = set({})
for val in [set(val) for val in train_data['Sequence']]:
    all_char_set = all_char_set.union(val)
all_char_list = list(all_char_set)
all_char_list.sort()
all_char_dict = {}
for i in range(len(all_char_list)):
    all_char_dict[all_char_list[i]] = i
    
##################################################################################
##### Create OHE of sequence
##################################################################################
train_data['OHE_Sequence'] = pd.Series([one_hot_encode_nt(val, all_char_dict) 
                                        for val in train_data["Sequence"]])

##################################################################################
##### Fix the labels
##################################################################################
train_data['label'] = pd.Series([1 if val == 1 else 0 
                                 for val in train_data["label_original"]])

##################################################################################
##### Extract features and labels, create folds
##################################################################################

features = np.array(list(train_data['OHE_Sequence']))
labels = np.array(list(train_data['label']))
labels = labels.reshape((labels.shape[0], 1))

input_seq_shape = features[0].shape

folds = build_kfold(features, labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

In [12]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    model = DLNN_CORENup(input_seq_shape = input_seq_shape)
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                           monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(fold["X_train"].shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model.fit(x = fold["X_train"][index_arr], y = fold["y_train"][index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
    
    model = tf.keras.models.load_model(current_model_path)
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)
    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)


Train/Test model on Fold #0.
Epoch 1/100
60/60 [==============================] - ETA: 0s - loss: 0.9125
Epoch 1: val_loss improved from inf to 0.88334, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
60/60 [==============================] - 13s 18ms/step - loss: 0.9125 - val_loss: 0.8833
Epoch 2/100
59/60 [============================>.] - ETA: 0s - loss: 0.8689
Epoch 2: val_loss improved from 0.88334 to 0.84539, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
60/60 [==============================] - 1s 10ms/step - loss: 0.8685 - val_loss: 0.8454
Epoch 3/100
53/60 [=========================>....] - ETA: 0s - loss: 0.8349
Epoch 3: val_loss improved from 0.84539 to 0.80098, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
60/60 [==============================] - 1s 10ms/step - loss: 0.8319 - val_loss: 0.8010
Epoch 4/10

60/60 [==============================] - 1s 10ms/step - loss: 0.4861 - val_loss: 0.5161
Epoch 27/100
55/60 [==========================>...] - ETA: 0s - loss: 0.4862
Epoch 27: val_loss improved from 0.51608 to 0.51548, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
60/60 [==============================] - 1s 10ms/step - loss: 0.4850 - val_loss: 0.5155
Epoch 28/100
60/60 [==============================] - ETA: 0s - loss: 0.4768
Epoch 28: val_loss did not improve from 0.51548
60/60 [==============================] - 1s 8ms/step - loss: 0.4768 - val_loss: 0.5221
Epoch 29/100
59/60 [============================>.] - ETA: 0s - loss: 0.4579
Epoch 29: val_loss did not improve from 0.51548
60/60 [==============================] - 1s 8ms/step - loss: 0.4591 - val_loss: 0.5239
Epoch 30/100
59/60 [============================>.] - ETA: 0s - loss: 0.4757
Epoch 30: val_loss improved from 0.51548 to 0.50969, saving model to Results\NT_Site_Pre

Epoch 59/100
60/60 [==============================] - ETA: 0s - loss: 0.3806
Epoch 59: val_loss did not improve from 0.46941
60/60 [==============================] - 1s 8ms/step - loss: 0.3806 - val_loss: 0.4736
Epoch 60/100
54/60 [==========================>...] - ETA: 0s - loss: 0.3624
Epoch 60: val_loss did not improve from 0.46941
60/60 [==============================] - 0s 8ms/step - loss: 0.3622 - val_loss: 0.4777
Epoch 61/100
58/60 [============================>.] - ETA: 0s - loss: 0.3691
Epoch 61: val_loss did not improve from 0.46941
60/60 [==============================] - 1s 9ms/step - loss: 0.3686 - val_loss: 0.4784
Epoch 62/100
58/60 [============================>.] - ETA: 0s - loss: 0.3613
Epoch 62: val_loss did not improve from 0.46941
60/60 [==============================] - 1s 9ms/step - loss: 0.3639 - val_loss: 0.4815
Epoch 63/100
58/60 [============================>.] - ETA: 0s - loss: 0.3514
Epoch 63: val_loss did not improve from 0.46941
60/60 [====================

57/60 [===========================>..] - ETA: 0s - loss: 0.3055
Epoch 97: val_loss did not improve from 0.46420
60/60 [==============================] - 0s 8ms/step - loss: 0.3024 - val_loss: 0.4850
Epoch 98/100
55/60 [==========================>...] - ETA: 0s - loss: 0.2990
Epoch 98: val_loss did not improve from 0.46420
60/60 [==============================] - 0s 8ms/step - loss: 0.2950 - val_loss: 0.4825
Epoch 99/100
60/60 [==============================] - ETA: 0s - loss: 0.3214
Epoch 99: val_loss did not improve from 0.46420
60/60 [==============================] - 0s 8ms/step - loss: 0.3214 - val_loss: 0.5142
Epoch 100/100
54/60 [==========================>...] - ETA: 0s - loss: 0.3099
Epoch 100: val_loss did not improve from 0.46420
60/60 [==============================] - 1s 8ms/step - loss: 0.3051 - val_loss: 0.5012

Train/Test model on Fold #1.
Epoch 1/100
55/60 [==========================>...] - ETA: 0s - loss: 0.9221
Epoch 1: val_loss improved from inf to 0.89171, saving mo

60/60 [==============================] - 1s 9ms/step - loss: 0.5014 - val_loss: 0.5221
Epoch 25/100
57/60 [===========================>..] - ETA: 0s - loss: 0.4914
Epoch 25: val_loss improved from 0.52213 to 0.51631, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold1.hdf5
60/60 [==============================] - 1s 9ms/step - loss: 0.4892 - val_loss: 0.5163
Epoch 26/100
57/60 [===========================>..] - ETA: 0s - loss: 0.4755
Epoch 26: val_loss improved from 0.51631 to 0.51487, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold1.hdf5
60/60 [==============================] - 1s 9ms/step - loss: 0.4763 - val_loss: 0.5149
Epoch 27/100
53/60 [=========================>....] - ETA: 0s - loss: 0.4830
Epoch 27: val_loss improved from 0.51487 to 0.51307, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold1.hdf5
60/60 [===========================

...

55/60 [==========================>...] - ETA: 0s - loss: 0.3658
Epoch 57: val_loss did not improve from 0.50655
60/60 [==============================] - 0s 8ms/step - loss: 0.3623 - val_loss: 0.5169
Epoch 58/100
55/60 [==========================>...] - ETA: 0s - loss: 0.3523
Epoch 58: val_loss did not improve from 0.50655
60/60 [==============================] - 0s 8ms/step - loss: 0.3540 - val_loss: 0.5198
Epoch 59/100
57/60 [===========================>..] - ETA: 0s - loss: 0.3539
Epoch 59: val_loss did not improve from 0.50655
60/60 [==============================] - 0s 8ms/step - loss: 0.3563 - val_loss: 0.5112
Epoch 60/100
56/60 [===========================>..] - ETA: 0s - loss: 0.3581
Epoch 60: val_loss did not improve from 0.50655
60/60 [==============================] - 0s 8ms/step - loss: 0.3516 - val_loss: 0.5258
Epoch 61/100
56/60 [===========================>..] - ETA: 0s - loss: 0.3621
Epoch 61: val_loss did not improve from 0.50655
60/60 [==============================] -

Epoch 96/100
58/60 [============================>.] - ETA: 0s - loss: 0.3083
Epoch 96: val_loss did not improve from 0.50655
60/60 [==============================] - 1s 9ms/step - loss: 0.3090 - val_loss: 0.5269
Epoch 97/100
57/60 [===========================>..] - ETA: 0s - loss: 0.2948
Epoch 97: val_loss did not improve from 0.50655
60/60 [==============================] - 1s 9ms/step - loss: 0.2985 - val_loss: 0.5147
Epoch 98/100
57/60 [===========================>..] - ETA: 0s - loss: 0.3119
Epoch 98: val_loss did not improve from 0.50655
60/60 [==============================] - 1s 9ms/step - loss: 0.3125 - val_loss: 0.5247
Epoch 99/100
59/60 [============================>.] - ETA: 0s - loss: 0.3050
Epoch 99: val_loss did not improve from 0.50655
60/60 [==============================] - 1s 9ms/step - loss: 0.3041 - val_loss: 0.5441
Epoch 100/100
54/60 [==========================>...] - ETA: 0s - loss: 0.3106
Epoch 100: val_loss did not improve from 0.50655
60/60 [==================

57/60 [===========================>..] - ETA: 0s - loss: 0.4647
Epoch 24: val_loss improved from 0.57894 to 0.57537, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold4.hdf5
60/60 [==============================] - 1s 10ms/step - loss: 0.4629 - val_loss: 0.5754
Epoch 25/100
57/60 [===========================>..] - ETA: 0s - loss: 0.4543
Epoch 25: val_loss improved from 0.57537 to 0.57282, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold4.hdf5
60/60 [==============================] - 1s 10ms/step - loss: 0.4539 - val_loss: 0.5728
Epoch 26/100
58/60 [============================>.] - ETA: 0s - loss: 0.4622
Epoch 26: val_loss improved from 0.57282 to 0.57099, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold4.hdf5
60/60 [==============================] - 1s 9ms/step - loss: 0.4622 - val_loss: 0.5710
Epoch 27/100
56/60 [=========================

58/60 [============================>.] - ETA: 0s - loss: 0.3443
Epoch 58: val_loss did not improve from 0.54262
60/60 [==============================] - 1s 9ms/step - loss: 0.3511 - val_loss: 0.5633
Epoch 59/100
60/60 [==============================] - ETA: 0s - loss: 0.3416
Epoch 59: val_loss did not improve from 0.54262
60/60 [==============================] - 1s 8ms/step - loss: 0.3416 - val_loss: 0.5686
Epoch 60/100
54/60 [==========================>...] - ETA: 0s - loss: 0.3330
Epoch 60: val_loss did not improve from 0.54262
60/60 [==============================] - 0s 8ms/step - loss: 0.3423 - val_loss: 0.5612
Epoch 61/100
58/60 [============================>.] - ETA: 0s - loss: 0.3440
Epoch 61: val_loss did not improve from 0.54262
60/60 [==============================] - 1s 9ms/step - loss: 0.3466 - val_loss: 0.5725
Epoch 62/100
57/60 [===========================>..] - ETA: 0s - loss: 0.3490
Epoch 62: val_loss did not improve from 0.54262
60/60 [==============================] -

Epoch 97/100
60/60 [==============================] - ETA: 0s - loss: 0.2944
Epoch 97: val_loss did not improve from 0.54262
60/60 [==============================] - 1s 8ms/step - loss: 0.2944 - val_loss: 0.5929
Epoch 98/100
59/60 [============================>.] - ETA: 0s - loss: 0.2924
Epoch 98: val_loss did not improve from 0.54262
60/60 [==============================] - 1s 9ms/step - loss: 0.2924 - val_loss: 0.5919
Epoch 99/100
56/60 [===========================>..] - ETA: 0s - loss: 0.2950
Epoch 99: val_loss did not improve from 0.54262
60/60 [==============================] - 1s 9ms/step - loss: 0.2953 - val_loss: 0.6213
Epoch 100/100
57/60 [===========================>..] - ETA: 0s - loss: 0.2725
Epoch 100: val_loss did not improve from 0.54262
60/60 [==============================] - 1s 9ms/step - loss: 0.2734 - val_loss: 0.6202


## k-fold Training evaluation

In [13]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.787562,0.789572,0.864722,0.784192,0.790915,0.575198
Train,0.936398,0.937209,0.978268,0.935566,0.937238,0.873133


In [14]:
evaluations_df[evaluations_df["Train_Test"] == "Test"]

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
1,0,Test,0.807128,0.804979,"[0.0, 0.0041841004184100415, 0.121338912133891...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9957843, 0.9957842, 0.9765404, 0.97560775, ...",0.886555,0.811715,0.802521,0.614269
3,1,Test,0.794549,0.804348,"[0.0, 0.004201680672268907, 0.1050420168067226...","[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...","[1.993231, 0.99323106, 0.963058, 0.9627936, 0....",0.869018,0.777311,0.811715,0.589400
5,2,Test,0.792017,0.790795,"[0.0, 0.004201680672268907, 0.0546218487394958...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9980549, 0.99805486, 0.98709166, 0.98684794...",0.866720,0.794118,0.789916,0.584039
7,3,Test,0.783613,0.787234,"[0.0, 0.004201680672268907, 0.0378151260504201...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9956019, 0.9956019, 0.98058367, 0.97834504,...",0.861000,0.777311,0.789916,0.567272
9,4,Test,0.760504,0.760504,"[0.0, 0.004201680672268907, 0.0420168067226890...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9967597, 0.9967596, 0.9828541, 0.9804451, 0...",0.840318,0.760504,0.760504,0.521008


# Independent data

In [15]:
train_features = features
train_labels = labels

In [16]:
##################################################################################
##### read independent data file
##################################################################################
indpe_file_path = os.path.join(input_data_folder, independent_data_file)
indpe_data = pd.read_csv(indpe_file_path, sep='\t', header=None)
indpe_data.columns = ['Sequence', 'name', 'id', 'flag', 'label_original', 'type']
indpe_data.head()
    
##################################################################################
##### Create OHE of sequence
##################################################################################
indpe_data['OHE_Sequence'] = pd.Series([one_hot_encode_nt(val, all_char_dict) 
                                        for val in indpe_data["Sequence"]])

##################################################################################
##### Fix the labels
##################################################################################
indpe_data['label'] = pd.Series([1 if val == 1 else 0 
                                 for val in indpe_data["label_original"]])

##################################################################################
##### Extract features and labels, create folds
##################################################################################

indpe_features = np.array(list(indpe_data['OHE_Sequence']))
indpe_labels = np.array(list(indpe_data['label']))
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

input_seq_shape = indpe_features[0].shape

## Using k-fold Models

### Performance of each k-fold model

In [17]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)
    mcc = matthews_corrcoef(indpe_labels, label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)

    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.62498,0.247755,0.673073,0.618719,0.626223,0.18509


In [18]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.603265,0.238447,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.00684931506849315, 0.006849315068...","[1.9970311, 0.9970311, 0.9890131, 0.9860087, 0...",0.680063,0.635468,0.596869,0.173966
1,1,Independent,0.645714,0.255814,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.003913894324853229, 0.00391389432...","[1.9967053, 0.99670523, 0.98629075, 0.98195994...",0.666307,0.596059,0.655577,0.192180
2,2,Independent,0.635102,0.251020,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.004892367906066536, 0.00489236790...","[1.9972383, 0.9972383, 0.98955977, 0.98942286,...",0.678598,0.605911,0.640900,0.187326
3,3,Independent,0.610612,0.240530,"[0.0, 0.0, 0.014778325123152709, 0.01477832512...","[0.0, 0.0009784735812133072, 0.000978473581213...","[1.9968368, 0.9968368, 0.9932454, 0.9921646, 0...",0.667483,0.625616,0.607632,0.175129
4,4,Independent,0.630204,0.252964,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0, 0.0, 0.005870841487279843, 0.005870...","[1.996693, 0.996693, 0.9912998, 0.98451334, 0....",0.672915,0.630542,0.630137,0.196852


### Mean score with k-fold models

In [19]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    total_pred += y_pred
    all_preds.append(y_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.636735,0.256048,0.685515,0.625616,0.638943,0.200397


### Voting score with k-fold models

In [20]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred
    all_preds.append(vote_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.636735,0.254065,0.662832,0.615764,0.6409,0.194672


## Using New Model

Train model on full data from training. Predict and evaluate on Independent data.

In [21]:
pos_indexes = np.where(indpe_labels==1)[0]
neg_indexes = np.random.permutation(np.where(indpe_labels==0)[0])[0:pos_indexes.shape[0]]
indpe_val_indexes = np.concatenate((pos_indexes, neg_indexes))

In [22]:
model = DLNN_CORENup(input_seq_shape = input_seq_shape)
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                       monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(train_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features[indpe_val_indexes], indpe_labels[indpe_val_indexes]))
# model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_split = 0.2)

model = tf.keras.models.load_model(current_model_path)

Epoch 1/100
75/75 [==============================] - ETA: 0s - loss: 0.9183
Epoch 1: val_loss improved from inf to 0.88429, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\_fullModel.hdf5
75/75 [==============================] - 3s 16ms/step - loss: 0.9183 - val_loss: 0.8843
Epoch 2/100
70/75 [===========================>..] - ETA: 0s - loss: 0.8732
Epoch 2: val_loss improved from 0.88429 to 0.84942, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\_fullModel.hdf5
75/75 [==============================] - 1s 9ms/step - loss: 0.8722 - val_loss: 0.8494
Epoch 3/100
69/75 [==========================>...] - ETA: 0s - loss: 0.8316
Epoch 3: val_loss improved from 0.84942 to 0.81245, saving model to Results\NT_Site_PredNTS_Classification_DLNN_CORENup_v2\5fold\models\_fullModel.hdf5
75/75 [==============================] - 1s 9ms/step - loss: 0.8303 - val_loss: 0.8124
Epoch 4/100
72/75 [===========================>..] - ETA: 

75/75 [==============================] - 1s 8ms/step - loss: 0.4551 - val_loss: 0.7204
Epoch 32/100
69/75 [==========================>...] - ETA: 0s - loss: 0.4365
Epoch 32: val_loss did not improve from 0.72039
75/75 [==============================] - 1s 8ms/step - loss: 0.4433 - val_loss: 0.7662
Epoch 33/100
69/75 [==========================>...] - ETA: 0s - loss: 0.4391
Epoch 33: val_loss did not improve from 0.72039
75/75 [==============================] - 1s 8ms/step - loss: 0.4388 - val_loss: 0.7781
Epoch 34/100
69/75 [==========================>...] - ETA: 0s - loss: 0.4285
Epoch 34: val_loss did not improve from 0.72039
75/75 [==============================] - 1s 8ms/step - loss: 0.4334 - val_loss: 0.7712
Epoch 35/100
75/75 [==============================] - ETA: 0s - loss: 0.4261
Epoch 35: val_loss did not improve from 0.72039
75/75 [==============================] - 1s 8ms/step - loss: 0.4261 - val_loss: 0.7760
Epoch 36/100
75/75 [==============================] - ETA: 0s - l

75/75 [==============================] - ETA: 0s - loss: 0.3644
Epoch 70: val_loss did not improve from 0.72039
75/75 [==============================] - 1s 8ms/step - loss: 0.3644 - val_loss: 0.8551
Epoch 71/100
69/75 [==========================>...] - ETA: 0s - loss: 0.3393
Epoch 71: val_loss did not improve from 0.72039
75/75 [==============================] - 1s 8ms/step - loss: 0.3386 - val_loss: 0.8710
Epoch 72/100
68/75 [==========================>...] - ETA: 0s - loss: 0.3583
Epoch 72: val_loss did not improve from 0.72039
75/75 [==============================] - 1s 8ms/step - loss: 0.3539 - val_loss: 0.8985
Epoch 73/100
74/75 [============================>.] - ETA: 0s - loss: 0.3579
Epoch 73: val_loss did not improve from 0.72039
75/75 [==============================] - 1s 8ms/step - loss: 0.3573 - val_loss: 0.8515
Epoch 74/100
72/75 [===========================>..] - ETA: 0s - loss: 0.3352
Epoch 74: val_loss did not improve from 0.72039
75/75 [==============================] -

In [23]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.607347,0.24635,0.676631,0.665025,0.59589,0.195114
